In [ ]:
# DeepSense Scenario 36 64 Beams: Model Fusion GPS, EffNetB0-B7, PointCloud!

In [ ]:
import os
import random
import datetime
import sys
import shutil

import torch
import torch as t
from torch.utils.data import Dataset, DataLoader
import torch.cuda as cuda
import torch.optim as optimizer
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transf
from torchvision import transforms, utils
from torchsummary import summary
from torch.utils.model_zoo import load_url as load_state_dict_from_url

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from skimage import io
import ast

import plotly.graph_objs as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from plyfile import PlyData, PlyElement

In [ ]:
# Main Model
# EfficientNet B0-B7

In [ ]:
from math import ceil

base_model = [
    # expand_ratio, channels, repeats, stride, kernel_size
    [1, 16, 1, 1, 3],
    [6, 24, 2, 2, 3],
    [6, 40, 2, 2, 5],
    [6, 80, 3, 2, 3],
    [6, 112, 3, 1, 5],
    [6, 192, 4, 2, 5],
    [6, 320, 1, 1, 3],
]

phi_values = {
    # tuple of: (phi_value, resolution, drop_rate)
    "b0": (0, 224, 0.2),  # alpha, beta, gamma, depth = alpha ** phi
    "b1": (0.5, 240, 0.2),
    "b2": (1, 260, 0.3),
    "b3": (2, 300, 0.3),
    "b4": (3, 380, 0.4),
    "b5": (4, 456, 0.4),
    "b6": (5, 528, 0.5),
    "b7": (6, 600, 0.5),
}


class CNNBlock(nn.Module):
    def __init__(
        self, in_channels, out_channels, kernel_size, stride, padding, groups=1
    ):
        super(CNNBlock, self).__init__()
        self.cnn = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            groups=groups,
            bias=False,
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.silu = nn.SiLU()  # SiLU <-> Swish

    def forward(self, x):
        return self.silu(self.bn(self.cnn(x)))


class SqueezeExcitation(nn.Module):
    def __init__(self, in_channels, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),  # C x H x W -> C x 1 x 1
            nn.Conv2d(in_channels, reduced_dim, 1),
            nn.SiLU(),
            nn.Conv2d(reduced_dim, in_channels, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.se(x)


class InvertedResidualBlock(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        stride,
        padding,
        expand_ratio,
        reduction=4,  # squeeze excitation
        survival_prob=0.8,  # for stochastic depth
    ):
        super(InvertedResidualBlock, self).__init__()
        self.survival_prob = 0.8
        self.use_residual = in_channels == out_channels and stride == 1
        hidden_dim = in_channels * expand_ratio
        self.expand = in_channels != hidden_dim
        reduced_dim = int(in_channels / reduction)

        if self.expand:
            self.expand_conv = CNNBlock(
                in_channels,
                hidden_dim,
                kernel_size=3,
                stride=1,
                padding=1,
            )

        self.conv = nn.Sequential(
            CNNBlock(
                hidden_dim,
                hidden_dim,
                kernel_size,
                stride,
                padding,
                groups=hidden_dim,
            ),
            SqueezeExcitation(hidden_dim, reduced_dim),
            nn.Conv2d(hidden_dim, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
        )

    def stochastic_depth(self, x):
        if not self.training:
            return x

        binary_tensor = (
            torch.rand(x.shape[0], 1, 1, 1, device=x.device) < self.survival_prob
        )
        return torch.div(x, self.survival_prob) * binary_tensor

    def forward(self, inputs):
        x = self.expand_conv(inputs) if self.expand else inputs

        if self.use_residual:
            return self.stochastic_depth(self.conv(x)) + inputs
        else:
            return self.conv(x)


class EfficientNet(nn.Module):
    def __init__(self, version):
        super(EfficientNet, self).__init__()
        width_factor, depth_factor, dropout_rate = self.calculate_factors(version)
        last_channels = ceil(1280 * width_factor)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.features = self.create_features(width_factor, depth_factor, last_channels)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(last_channels, 128),
        )

    def calculate_factors(self, version, alpha=1.2, beta=1.1):
        phi, res, drop_rate = phi_values[version]
        depth_factor = alpha**phi
        width_factor = beta**phi
        return width_factor, depth_factor, drop_rate

    def create_features(self, width_factor, depth_factor, last_channels):
        channels = int(32 * width_factor)
        features = [CNNBlock(3, channels, 3, stride=2, padding=1)]
        in_channels = channels

        for expand_ratio, channels, repeats, stride, kernel_size in base_model:
            out_channels = 4 * ceil(int(channels * width_factor) / 4)
            layers_repeats = ceil(repeats * depth_factor)

            for layer in range(layers_repeats):
                features.append(
                    InvertedResidualBlock(
                        in_channels,
                        out_channels,
                        expand_ratio=expand_ratio,
                        stride=stride if layer == 0 else 1,
                        kernel_size=kernel_size,
                        padding=kernel_size // 2,  # if k=1:pad=0, k=3:pad=1, k=5:pad=2
                    )
                )
                in_channels = out_channels

        features.append(
            CNNBlock(in_channels, last_channels, kernel_size=1, stride=1, padding=0)
        )

        return nn.Sequential(*features)

    def forward(self, x):
        x = self.pool(self.features(x))
        return self.classifier(x.view(x.shape[0], -1))

In [ ]:
# Proposed CNN Model for GPS

class NN_beam_pred(nn.Module):
    def __init__(self, num_features):
        super(NN_beam_pred, self).__init__()


        self.cnn_layers = nn.Sequential(
            # Defining the convolution layer
            nn.Conv1d(num_features, 30, kernel_size=2, stride=1, padding=1),
            nn.Conv1d(30, 20, kernel_size=2, stride=1, padding=1),
            nn.Conv1d(20, 10, kernel_size=2, stride=1, padding=1),
            nn.MaxPool1d(kernel_size=2, stride=2),

            nn.Conv1d(10, 30, kernel_size=2, stride=1, padding=1),
            nn.Conv1d(30, 20, kernel_size=2, stride=1, padding=1),
            nn.Conv1d(20, 10, kernel_size=2, stride=1, padding=1),
            nn.MaxPool1d(kernel_size=2, stride=2),

            nn.Conv1d(10, 30, kernel_size=2, stride=1, padding=1),
            nn.Conv1d(30, 20, kernel_size=2, stride=1, padding=1),
            nn.Conv1d(20, 10, kernel_size=2, stride=1, padding=1),
            nn.MaxPool1d(kernel_size=2, stride=2),

        )

        self.l1 = nn.Linear(20, 128)
        self.l2 = nn.Linear(128, 128)
        # self.l3 = nn.Linear(128, num_output)

    def forward(self, inputs):
        x = self.cnn_layers(inputs)
        # print(x.shape)
        x = x.view(x.size(0), -1)
        x = self.l1(x)
        x = self.l2(x)
        # x = self.l3(x)
        return x

In [ ]:
# PointNet Model for LiDAR Data

class TNet(nn.Module):
    def __init__(self, k=3):
        super(TNet, self).__init__()
        self.conv1 = nn.Conv1d(k, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 15000*k)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        # identity_matrix = torch.eye(15000, device=x.device).view(1, 3, 15000).repeat(x.size(0), 1, 1)
        x = x.view(-1, 15000, 3) #+ identity_matrix
        return x

class PointNet(nn.Module):
    def __init__(self, num_classes):
        super(PointNet, self).__init__()
        self.input_transform = TNet(k=3)  # Input shape is (3, 3)
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 1024, 1)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
        # Apply input transformation
        transformation = self.input_transform(x)
        x = torch.bmm(x, transformation)  # Apply the transformation to input points

        # PointNet layers
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 1024)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Training, Testing, and Validation!

In [ ]:
# Training Hyper-parameters
batch_size = 16
val_batch_size = 1
lr = 0.01
decay = 1e-4
num_epochs = 40
train_size = [1]

# Hyperparameters for our network
input_size = 2
node = 512
output_size = 65

In [ ]:
def create_samples(root, shuffle=False, nat_sort=False):
    f = pd.read_csv(root)
    data_samples = []
    pred_val = []
    for idx, row in f.iterrows():
        data = list(row.values[1:])
        data_samples.append(data)

    return data_samples


class DataFeed(Dataset):
    '''
    A class retrieving a tuple of (image,label). It can handle the case
    of empty classes (empty folders).
    '''
    def __init__(self, image_dir, pos_dir, lidar_dir, nat_sort = False, num_points=15000, image_transform=None, pos_transform = None, init_shuflle = True):
        self.root1 = image_dir
        self.root2 = pos_dir
        self.root3 = lidar_dir

        self.num_points = num_points

        self.samples1 = create_samples(self.root1, shuffle=init_shuflle, nat_sort=nat_sort)
        self.samples2 = create_samples(self.root2, shuffle=init_shuflle, nat_sort=nat_sort)
        self.samples3 = create_samples(self.root3, shuffle=init_shuflle, nat_sort=nat_sort)
        self.transform1 = image_transform
        self.transform2 = pos_transform


    def __len__(self):
        return len(self.samples1)

    def __getitem__(self, idx):
        sample1 = self.samples1[idx]
        img = io.imread(sample1[0])
        if self.transform1:
            img = self.transform1(img)
        label = sample1[1]

        ################################
        sample2 = self.samples2[idx]

        pos_val = sample2[:1]
        pos_val = ast.literal_eval(pos_val[0])
        pos_val = np.asarray(pos_val)
        ################################
        sample3 = self.samples3[idx]
        df = pd.read_csv(sample3[0])
        # Extract the columns of interest
        x = df[' X (mm)'] / 1000  # Convert to meters
        y = df[' Y (mm)'] / 1000
        z = df[' Z (mm)'] / 1000
        points = np.column_stack((x.values, y.values, z.values))
        if points.shape[0] < self.num_points:
            points = np.pad(points, ((0, self.num_points - points.shape[0]), (0, 0)), mode='constant')
        elif points.shape[0] > self.num_points:
            indices = np.random.choice(points.shape[0], self.num_points, replace=False)
            points = points[indices]

        points = torch.tensor(points, dtype=torch.float32)

        # pos_centers = sample2[1:2]
        # pos_centers = np.asarray(pos_centers)

        return ([img, pos_val, points.T], label)

In [ ]:
# Image transform
img_resize = transf.Resize((224, 224))
img_norm = transf.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
image_proc_pipe = transf.Compose(
    [transf.ToPILImage(),
     img_resize,
     transf.ToTensor(),
     img_norm]
)


#POS transform
pos_proc_pipe = transf.Compose(
    [
        transf.ToTensor()
    ]
)

In [ ]:
# year month day
dayTime = datetime.datetime.now().strftime('%m-%d-%Y')
# Minutes and seconds
hourTime = datetime.datetime.now().strftime('%H_%M')
print(dayTime + '\n' + hourTime)

In [ ]:
pwd = os.getcwd() + '//' + 'saved_folder' + '//' + dayTime + '_' + hourTime
print(pwd)
# Determine whether the folder already exists
isExists = os.path.exists(pwd)
if not isExists:
    os.makedirs(pwd)

In [ ]:
# Copy the training files to the saved directory
shutil.copy('./scenario36_64_img_beam_train.csv', pwd)
shutil.copy('./scenario36_64_img_beam_val.csv', pwd)
shutil.copy('./scenario36_64_img_beam_test.csv', pwd)
shutil.copy('./scenario36_64_lidar_beam_train.csv', pwd)
shutil.copy('./scenario36_64_lidar_beam_val.csv', pwd)
shutil.copy('./scenario36_64_lidar_beam_test.csv', pwd)
shutil.copy('./scenario36_64_pos_beam_train.csv', pwd)
shutil.copy('./scenario36_64_pos_beam_val.csv', pwd)
shutil.copy('./scenario36_64_pos_beam_test.csv', pwd)

In [ ]:
image_train_dir = './scenario36_64_img_beam_train.csv'
image_val_dir = './scenario36_64_img_beam_val.csv'

pos_train_dir = './scenario36_64_pos_beam_train.csv'
pos_val_dir = './scenario36_64_pos_beam_val.csv'

lidar_train_dir = './scenario36_64_lidar_beam_train.csv'
lidar_val_dir = './scenario36_64_lidar_beam_val.csv'

train_loader = DataLoader(DataFeed(image_train_dir, pos_train_dir, lidar_train_dir, image_transform=image_proc_pipe),
                          batch_size=batch_size,
                          #num_workers=8,
                          shuffle=False)
val_loader = DataLoader(DataFeed(image_val_dir, pos_val_dir, lidar_train_dir, image_transform=image_proc_pipe),
                        batch_size=val_batch_size,
                        #num_workers=8,
                        shuffle=False)

In [ ]:
save_directory = pwd + '//' + 'saved_analysis_files'
checkpoint_directory = pwd + '//' + 'checkpoint'

isExists = os.path.exists(save_directory)
if not isExists:
    os.makedirs(save_directory)

isExists = os.path.exists(checkpoint_directory)
if not isExists:
    os.makedirs(checkpoint_directory)

In [ ]:
class fusion_model(nn.Module):
    def __init__(self, version, num_features, num_classes):
        super(fusion_model, self).__init__()
        self.nclass = num_classes

        self.nf = num_features
        self.version = version

        self.eff = EfficientNet(self.version)
        self.cnv = NN_beam_pred(self.nf)
        self.pnet = PointNet(num_classes=self.nclass)

        self.fc1 = nn.Linear(321, 256)
        self.fc2 = nn.Linear(256, self.nclass)

    def forward(self, inputs):
        x = self.eff(inputs[0])
        y = self.cnv(inputs[1].reshape([-1, 2, 1]))
        z = self.pnet(inputs[2])

        x = torch.cat([x, y, z], dim=1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = fusion_model("b0", input_size, num_classes=output_size)

In [ ]:
val_acc = []

with cuda.device(0):
    top_1 = np.zeros( (1,len(train_size)) )
    top_3 = np.zeros( (1,len(train_size)) )
    top_5 = np.zeros( (1,len(train_size)) )
    top_7 = np.zeros( (1,len(train_size)) )
    top_9 = np.zeros( (1,len(train_size)) )
    top_11 = np.zeros( (1,len(train_size)) )
    top_13 = np.zeros( (1,len(train_size)) )
    top_15 = np.zeros( (1,len(train_size)) )
    acc_loss = 0
    itr = []
    for idx, n in enumerate(train_size):
        print('```````````````````````````````````````````````````````')
        print('Training size is {}'.format(n))
        # Build the network:
        net = model.cuda()
        layers = list(net.children())

        #  Optimization parameters:
        criterion = nn.CrossEntropyLoss()
        opt = optimizer.Adam(net.parameters(), lr=lr, weight_decay=decay)
        LR_sch = optimizer.lr_scheduler.MultiStepLR(opt, [15,25,40], gamma=0.1, last_epoch=-1)

        count = 0
        running_loss = []
        running_top1_acc = []
        running_top3_acc = []
        running_top5_acc = []
        running_top7_acc = []
        running_top9_acc = []
        running_top11_acc = []
        running_top13_acc = []
        running_top15_acc = []

        best_accuracy = 0

        for epoch in range(num_epochs):
            print('Epoch No. ' + str(epoch + 1))
            skipped_batches = 0
            for tr_count, (pos_data, beam_val) in enumerate(train_loader):
                net.train()
                data = [pos_data[0], pos_data[1].type(torch.Tensor), pos_data[2].type(torch.Tensor)]
                label = beam_val.type(torch.LongTensor)
                x = [data[0].cuda(), data[1].cuda(), data[2].cuda()]
                #print("x size", x.size())
                opt.zero_grad()
                label = label.cuda()
                #print("label size", label.size())
                # print(x.shape)
                out = net.forward(x)
                # print(out.shape, label.shape)
                loss = criterion(out, label)
                loss.backward()
                opt.step()
                batch_loss = loss.item()
                acc_loss += batch_loss
                count += 1
                if np.mod(count, 100) == 0:
                    print('Training-Batch No.' + str(count))
                    running_loss.append(batch_loss)  # running_loss.append()
                    itr.append(count)
                    print('Loss = ' + str(running_loss[-1]))

            print('Start Validation')
            ave_top1_acc = 0
            ave_top3_acc = 0
            ave_top5_acc = 0
            ave_top7_acc = 0
            ave_top9_acc = 0
            ave_top11_acc = 0
            ave_top13_acc = 0
            ave_top15_acc = 0
            ind_ten = t.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')
            top1_pred_out = []
            top3_pred_out = []
            top5_pred_out = []
            top7_pred_out = []
            top9_pred_out = []
            top11_pred_out = []
            top13_pred_out = []
            top15_pred_out = []
            total_count = 0

            gt_beam = []
            for val_count, (pos_data, beam_val) in enumerate(val_loader):
                net.eval()
                data = [pos_data[0], pos_data[1].type(torch.Tensor), pos_data[2].type(torch.Tensor)]
                label = beam_val.type(torch.LongTensor)
                x = [data[0].cuda(), data[1].cuda(), data[2].cuda()]
                #print("x size", x.size())
                opt.zero_grad()
                labels = label.cuda()
                gt_beam.append(labels.detach().cpu().numpy()[0].tolist())
                total_count += labels.size(0)
                out = net.forward(x)
                _, top_1_pred = t.max(out, dim=1)
                top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0].tolist())
                sorted_out = t.argsort(out, dim=1, descending=True)

                top_3_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:3])
                top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0].tolist())

                top_5_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:5])
                top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0].tolist())

                top_7_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:7])
                top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0].tolist())

                top_9_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:9])
                top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0].tolist())

                top_11_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:11])
                top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0].tolist())

                top_13_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:13])
                top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0].tolist())

                top_15_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:15])
                top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0].tolist())

                reshaped_labels = labels.reshape((labels.shape[0], 1))
                tiled_3_labels = reshaped_labels.repeat(1, 3)
                tiled_5_labels = reshaped_labels.repeat(1, 5)
                tiled_7_labels = reshaped_labels.repeat(1, 7)
                tiled_9_labels = reshaped_labels.repeat(1, 9)
                tiled_11_labels = reshaped_labels.repeat(1, 11)
                tiled_13_labels = reshaped_labels.repeat(1, 13)
                tiled_15_labels = reshaped_labels.repeat(1, 15)

                batch_top1_acc = t.sum(top_1_pred == labels, dtype=t.float32)
                batch_top3_acc = t.sum(top_3_pred == tiled_3_labels, dtype=t.float32)
                batch_top5_acc = t.sum(top_5_pred == tiled_5_labels, dtype=t.float32)
                batch_top7_acc = t.sum(top_7_pred == tiled_7_labels, dtype=t.float32)
                batch_top9_acc = t.sum(top_9_pred == tiled_9_labels, dtype=t.float32)
                batch_top11_acc = t.sum(top_11_pred == tiled_11_labels, dtype=t.float32)
                batch_top13_acc = t.sum(top_13_pred == tiled_13_labels, dtype=t.float32)
                batch_top15_acc = t.sum(top_15_pred == tiled_15_labels, dtype=t.float32)

                ave_top1_acc += batch_top1_acc.item()
                ave_top3_acc += batch_top3_acc.item()
                ave_top5_acc += batch_top5_acc.item()
                ave_top7_acc += batch_top7_acc.item()
                ave_top9_acc += batch_top9_acc.item()
                ave_top11_acc += batch_top11_acc.item()
                ave_top13_acc += batch_top13_acc.item()
                ave_top15_acc += batch_top15_acc.item()

            print("total test examples are", total_count)
            running_top1_acc.append(ave_top1_acc / total_count)  # (batch_size * (count_2 + 1)) )
            running_top3_acc.append(ave_top3_acc / total_count)
            running_top5_acc.append(ave_top5_acc / total_count)
            running_top7_acc.append(ave_top7_acc / total_count)
            running_top9_acc.append(ave_top9_acc / total_count)
            running_top11_acc.append(ave_top11_acc / total_count)
            running_top13_acc.append(ave_top13_acc / total_count)
            running_top15_acc.append(ave_top15_acc / total_count)

            print('Training_size {}--No. of skipped batchess {}'.format(n,skipped_batches))
            print('Average Top-1 accuracy {}'.format( running_top1_acc[-1]))
            print('Average Top-3 accuracy {}'.format( running_top3_acc[-1]))
            print('Average Top-5 accuracy {}'.format( running_top5_acc[-1]))
            print('Average Top-7 accuracy {}'.format( running_top7_acc[-1]))
            print('Average Top-9 accuracy {}'.format( running_top9_acc[-1]))
            print('Average Top-11 accuracy {}'.format( running_top11_acc[-1]))
            print('Average Top-13 accuracy {}'.format( running_top13_acc[-1]))
            print('Average Top-15 accuracy {}'.format( running_top15_acc[-1]))

            cur_accuracy  = running_top1_acc[-1]

            print("current acc", cur_accuracy)
            print("best acc", best_accuracy)
            if cur_accuracy > best_accuracy:
                print("Saving the best model")
                net_name = checkpoint_directory  + '//' +  'fused_nn_beam_pred'
                t.save(net.state_dict(), net_name)
                best_accuracy =  cur_accuracy
            print("updated best acc", best_accuracy)


            print("Saving the predicted value in a csv file")
            file_to_save = f'{save_directory}//topk_fused_val_after_{epoch+1}th_epoch.csv'
            indx = np.arange(1, len(top1_pred_out)+1, 1)
            df1 = pd.DataFrame()
            df1['index'] = indx
            df1['link_status'] = gt_beam
            df1['top1_pred'] = top1_pred_out
            df1['top3_pred'] = top3_pred_out
            df1['top5_pred'] = top5_pred_out
            df1['top7_pred'] = top7_pred_out
            df1['top9_pred'] = top9_pred_out
            df1['top11_pred'] = top11_pred_out
            df1['top13_pred'] = top13_pred_out
            df1['top15_pred'] = top15_pred_out
            df1.to_csv(file_to_save, index=False)

            LR_sch.step()

        top_1[0,idx] = running_top1_acc[-1]
        top_3[0,idx] = running_top3_acc[-1]
        top_5[0,idx] = running_top5_acc[-1]
        top_7[0,idx] = running_top7_acc[-1]
        top_9[0,idx] = running_top9_acc[-1]
        top_11[0,idx] = running_top11_acc[-1]
        top_13[0,idx] = running_top13_acc[-1]
        top_15[0,idx] = running_top15_acc[-1]

In [ ]:
print('Start Validation')
ave_top1_acc = 0
ave_top3_acc = 0
ave_top5_acc = 0
ave_top7_acc = 0
ave_top9_acc = 0
ave_top11_acc = 0
ave_top13_acc = 0
ave_top15_acc = 0
ind_ten = t.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')
top1_pred_out = []
top3_pred_out = []
top5_pred_out = []
top7_pred_out = []
top9_pred_out = []
top11_pred_out = []
top13_pred_out = []
top15_pred_out = []
running_top1_acc = []
running_top3_acc = []
running_top5_acc = []
running_top7_acc = []
running_top9_acc = []
running_top11_acc = []
running_top13_acc = []
running_top15_acc = []
total_count = 0

gt_beam = []

for val_count, (pos_data, beam_val) in enumerate(val_loader):
    net.eval()
    data = [pos_data[0], pos_data[1].type(torch.Tensor), pos_data[2].type(torch.Tensor)]
    label = beam_val.type(torch.LongTensor)
    x = [data[0].cuda(), data[1].cuda(), data[2].cuda()]
    #print("x size", x.size())
    opt.zero_grad()
    labels = label.cuda()

    gt_beam.append(labels.detach().cpu().numpy()[0].tolist())
    total_count += labels.size(0)
    out = net.forward(x)
    _, top_1_pred = t.max(out, dim=1)
    top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0].tolist())
    sorted_out = t.argsort(out, dim=1, descending=True)

    top_3_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:3])
    top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0].tolist())

    top_5_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:5])
    top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0].tolist())

    top_7_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:7])
    top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0].tolist())

    top_9_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:9])
    top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0].tolist())

    top_11_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:11])
    top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0].tolist())

    top_13_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:13])
    top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0].tolist())

    top_15_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:15])
    top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0].tolist())

    reshaped_labels = labels.reshape((labels.shape[0], 1))
    tiled_3_labels = reshaped_labels.repeat(1, 3)
    tiled_5_labels = reshaped_labels.repeat(1, 5)
    tiled_7_labels = reshaped_labels.repeat(1, 7)
    tiled_9_labels = reshaped_labels.repeat(1, 9)
    tiled_11_labels = reshaped_labels.repeat(1, 11)
    tiled_13_labels = reshaped_labels.repeat(1, 13)
    tiled_15_labels = reshaped_labels.repeat(1, 15)

    batch_top1_acc = t.sum(top_1_pred == labels, dtype=t.float32)
    batch_top3_acc = t.sum(top_3_pred == tiled_3_labels, dtype=t.float32)
    batch_top5_acc = t.sum(top_5_pred == tiled_5_labels, dtype=t.float32)
    batch_top7_acc = t.sum(top_7_pred == tiled_7_labels, dtype=t.float32)
    batch_top9_acc = t.sum(top_9_pred == tiled_9_labels, dtype=t.float32)
    batch_top11_acc = t.sum(top_11_pred == tiled_11_labels, dtype=t.float32)
    batch_top13_acc = t.sum(top_13_pred == tiled_13_labels, dtype=t.float32)
    batch_top15_acc = t.sum(top_15_pred == tiled_15_labels, dtype=t.float32)

    ave_top1_acc += batch_top1_acc.item()
    ave_top3_acc += batch_top3_acc.item()
    ave_top5_acc += batch_top5_acc.item()
    ave_top7_acc += batch_top7_acc.item()
    ave_top9_acc += batch_top9_acc.item()
    ave_top11_acc += batch_top11_acc.item()
    ave_top13_acc += batch_top13_acc.item()
    ave_top15_acc += batch_top15_acc.item()

print("total examples are", total_count)
running_top1_acc.append(ave_top1_acc / total_count)  # (batch_size * (count_2 + 1)) )
running_top3_acc.append(ave_top3_acc / total_count)
running_top5_acc.append(ave_top5_acc / total_count)
running_top7_acc.append(ave_top7_acc / total_count)
running_top9_acc.append(ave_top9_acc / total_count)
running_top11_acc.append(ave_top11_acc / total_count)
running_top13_acc.append(ave_top13_acc / total_count)
running_top15_acc.append(ave_top15_acc / total_count)

print('Training_size {}--No. of skipped batchess {}'.format(n,skipped_batches))
print('Average Top-1 accuracy {}'.format( running_top1_acc[-1]))
print('Average Top-3 accuracy {}'.format( running_top3_acc[-1]))
print('Average Top-5 accuracy {}'.format( running_top5_acc[-1]))
print('Average Top-7 accuracy {}'.format( running_top7_acc[-1]))
print('Average Top-9 accuracy {}'.format( running_top9_acc[-1]))
print('Average Top-11 accuracy {}'.format( running_top11_acc[-1]))
print('Average Top-13 accuracy {}'.format( running_top13_acc[-1]))
print('Average Top-15 accuracy {}'.format( running_top15_acc[-1]))

print("Saving the predicted value in a csv file")
file_to_save = f'{save_directory}//best_epoch_eval.csv'

indx = np.arange(1, len(top1_pred_out)+1, 1)
df2 = pd.DataFrame()
df2['index'] = indx
df2['link_status'] = gt_beam  # Add the link_status column

df2['top1_pred'] = top1_pred_out
df2['top3_pred'] = top3_pred_out
df2['top5_pred'] = top5_pred_out
df2['top7_pred'] = top7_pred_out
df2['top9_pred'] = top9_pred_out
df2['top11_pred'] = top11_pred_out
df2['top13_pred'] = top13_pred_out
df2['top15_pred'] = top15_pred_out
df2.to_csv(file_to_save, index=False)

In [ ]:
# Testing, need to work from here!

In [ ]:
# Load the model checkpoint
image_test_dir = './scenario36_64_img_beam_test.csv'
pos_test_dir = './scenario36_64_pos_beam_test.csv'
lidar_test_dir = './scenario36_64_lidar_beam_test.csv'

# Load the test data
test_data = pd.read_csv(image_test_dir)

# Extract the 'unit1_pwr1_best-beam' data and convert it to a list
link_status_data = test_data['original_unit1_pwr3_best-beam'].tolist()
org = test_data['original_index'].tolist()
pwr_60ghz = test_data['original_unit1_pwr3'].tolist()

checkpoint_path = f'{checkpoint_directory}/fused_nn_beam_pred'
model.load_state_dict(torch.load(checkpoint_path))
model.eval()
net = model.cuda()

In [ ]:
test_loader = DataLoader(DataFeed(image_test_dir, pos_test_dir, lidar_test_dir, image_transform=image_proc_pipe),
                            batch_size=val_batch_size,
                            #num_workers=8,
                            shuffle=False)

In [ ]:
print('Start Testing')
ave_top1_acc = 0
ave_top3_acc = 0
ave_top5_acc = 0
ave_top7_acc = 0
ave_top9_acc = 0
ave_top11_acc = 0
ave_top13_acc = 0
ave_top15_acc = 0
ind_ten = t.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')
top1_pred_out = []
top3_pred_out = []
top5_pred_out = []
top7_pred_out = []
top9_pred_out = []
top11_pred_out = []
top13_pred_out = []
top15_pred_out = []
running_top1_acc = []
running_top3_acc = []
running_top5_acc = []
running_top7_acc = []
running_top9_acc = []
running_top11_acc = []
running_top13_acc = []
running_top15_acc = []
total_count = 0

gt_beam = []

for val_count, (pos_data, beam_val) in enumerate(test_loader):
    net.eval()
    data = [pos_data[0], pos_data[1].type(torch.Tensor), pos_data[2].type(torch.Tensor)]
    label = beam_val.type(torch.LongTensor)
    x = [data[0].cuda(), data[1].cuda(), data[2].cuda()]
    #print("x size", x.size())
    opt.zero_grad()
    labels = label.cuda()

    gt_beam.append(labels.detach().cpu().numpy()[0].tolist())
    total_count += labels.size(0)
    out = net.forward(x)
    _, top_1_pred = t.max(out, dim=1)
    top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0].tolist())
    sorted_out = t.argsort(out, dim=1, descending=True)

    top_3_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:3])
    top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0].tolist())

    top_5_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:5])
    top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0].tolist())

    top_7_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:7])
    top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0].tolist())

    top_9_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:9])
    top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0].tolist())

    top_11_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:11])
    top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0].tolist())

    top_13_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:13])
    top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0].tolist())

    top_15_pred = t.index_select(sorted_out, dim=1, index=ind_ten[0:15])
    top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0].tolist())

    reshaped_labels = labels.reshape((labels.shape[0], 1))
    tiled_3_labels = reshaped_labels.repeat(1, 3)
    tiled_5_labels = reshaped_labels.repeat(1, 5)
    tiled_7_labels = reshaped_labels.repeat(1, 7)
    tiled_9_labels = reshaped_labels.repeat(1, 9)
    tiled_11_labels = reshaped_labels.repeat(1, 11)
    tiled_13_labels = reshaped_labels.repeat(1, 13)
    tiled_15_labels = reshaped_labels.repeat(1, 15)

    batch_top1_acc = t.sum(top_1_pred == labels, dtype=t.float32)
    batch_top3_acc = t.sum(top_3_pred == tiled_3_labels, dtype=t.float32)
    batch_top5_acc = t.sum(top_5_pred == tiled_5_labels, dtype=t.float32)
    batch_top7_acc = t.sum(top_7_pred == tiled_7_labels, dtype=t.float32)
    batch_top9_acc = t.sum(top_9_pred == tiled_9_labels, dtype=t.float32)
    batch_top11_acc = t.sum(top_11_pred == tiled_11_labels, dtype=t.float32)
    batch_top13_acc = t.sum(top_13_pred == tiled_13_labels, dtype=t.float32)
    batch_top15_acc = t.sum(top_15_pred == tiled_15_labels, dtype=t.float32)

    ave_top1_acc += batch_top1_acc.item()
    ave_top3_acc += batch_top3_acc.item()
    ave_top5_acc += batch_top5_acc.item()
    ave_top7_acc += batch_top7_acc.item()
    ave_top9_acc += batch_top9_acc.item()
    ave_top11_acc += batch_top11_acc.item()
    ave_top13_acc += batch_top13_acc.item()
    ave_top15_acc += batch_top15_acc.item()

print("total examples are", total_count)
running_top1_acc.append(ave_top1_acc / total_count)  # (batch_size * (count_2 + 1)) )
running_top3_acc.append(ave_top3_acc / total_count)
running_top5_acc.append(ave_top5_acc / total_count)
running_top7_acc.append(ave_top7_acc / total_count)
running_top9_acc.append(ave_top9_acc / total_count)
running_top11_acc.append(ave_top11_acc / total_count)
running_top13_acc.append(ave_top13_acc / total_count)
running_top15_acc.append(ave_top15_acc / total_count)

print('Training_size {}--No. of skipped batchess {}'.format(n,skipped_batches))
print('Average Top-1 accuracy {}'.format( running_top1_acc[-1]))
print('Average Top-3 accuracy {}'.format( running_top3_acc[-1]))
print('Average Top-5 accuracy {}'.format( running_top5_acc[-1]))
print('Average Top-7 accuracy {}'.format( running_top7_acc[-1]))
print('Average Top-9 accuracy {}'.format( running_top9_acc[-1]))
print('Average Top-11 accuracy {}'.format( running_top11_acc[-1]))
print('Average Top-13 accuracy {}'.format( running_top13_acc[-1]))
print('Average Top-15 accuracy {}'.format( running_top15_acc[-1]))

print("Saving the predicted value in a csv file")
file_to_save = f'{save_directory}//best_epoch_Test.csv'


# Extract the 'unit1_pwr1_best-beam' data and convert it to a list
# link_status_data = test_data['original_unit1_pwr3_best-beam'].tolist()
# org = test_data['original_index'].tolist()
# pwr_60ghz = test_data['original_unit1_pwr3'].tolist()

indx = test_data.index + 1
df2 = pd.DataFrame()
df2['index'] = org
df2['link_status'] = link_status_data  # Add the link_status column
df2['original_unit1_pwr3'] = pwr_60ghz # Add the original_unit1_pwr_60ghz column

df2['top1_pred'] = top1_pred_out
df2['top3_pred'] = top3_pred_out
df2['top5_pred'] = top5_pred_out
df2['top7_pred'] = top7_pred_out
df2['top9_pred'] = top9_pred_out
df2['top11_pred'] = top11_pred_out
df2['top13_pred'] = top13_pred_out
df2['top15_pred'] = top15_pred_out
df2.to_csv(file_to_save, index=False)